In [2]:
import pandas as pd
import pickle as pkl
import mysql.connector
from tqdm.notebook import tqdm
from collections import Counter
from random import shuffle
from pymetamap import MetaMap

In [2]:
(dict_defs, dict_relations, dict_semantic_types, 
          loinc_cuis, all_key_based_cuis, 
          loinc_depth_1, key_depth_1, 
          loinc_depth_2, key_depth_2,
          family_cuis, loinc_filtered_cuis,
          key_filtered_cuis, key_filtered_cuis_reduced,
          dict_og_category_assignment, dict_va_categories,
          selected_cuis_after_reduced_filtering
         ) = pkl.load(open('../data/cui_april_20th_1.pkl', 'rb'))

In [3]:
selected_cuis_after_reduced_filtering.__len__()

198677

In [4]:
type(selected_cuis_after_reduced_filtering)

set

In [5]:
for each_cui in selected_cuis_after_reduced_filtering: 
    if each_cui not in dict_defs:
        exit_

In [13]:
for each_cui in selected_cuis_after_reduced_filtering: 
    if each_cui not in dict_semantic_types:
        exit_

In [11]:
sum([dict_og_category_assignment[x].__len__() for x in dict_og_category_assignment])

132163

In [6]:
each_cui

'C0430578'

In [14]:
dict_defs[each_cui].head()

,CUI,LAT,TS,LUI,STT,SUI,ISPREF,AUI,SAUI,SCUI,SDUI,SAB,TTY,CODE,STR,SRL,SUPPRESS,CVF
0,C0430578,ENG,P,L0878795,PF,S0989706,N,A1047893,None,None,None,RCD,PT,X77eF,Photo patch test,3,N,256.0
1,C0430578,ENG,S,L7046589,PF,S8733141,Y,A13612548,None,18454,None,MEDCIN,FN,18454,allergy testing photo patch tests (lab test),3,N,NaN
2,C0430578,ENG,S,L7193144,PF,S8733152,Y,A13648045,None,18454,None,MEDCIN,PT,18454,allergy testing photo patch tests,3,N,NaN
3,C0430578,ENG,P,L0878795,VO,S9146653,Y,A15298241,None,18454,None,MEDCIN,SY,18454,photo patch tests,3,N,NaN
4,C0430578,ENG,S,L0875356,PF,S1799820,Y,A1751404,None,95052,None,CPT,PT,95052,Photo patch test(s) (specify number of tests),3,N,NaN


In [16]:
dict_semantic_types[each_cui]

(        CUI   TUI       STN                   STY        ATUI  CVF
 0  C0430578  T060  B1.3.1.2  Diagnostic Procedure  AT89418668  256,
 {'Diagnostic Procedure'})

In [17]:
dict_final = {}
dict_final['cuis'] = selected_cuis_after_reduced_filtering
dict_final['cui_defs'] = {}
dict_final['cui_stys'] = {}
for each_cui in tqdm(dict_final['cuis']):
    dict_final['cui_defs'][each_cui] = dict_defs[each_cui]['STR'].tolist()
    dict_final['cui_stys'][each_cui] = dict_semantic_types[each_cui][-1]

  0%|          | 0/198677 [00:00<?, ?it/s]

In [19]:
dict_final['cui_defs']['C0430578']

['Photo patch test',
 'allergy testing photo patch tests (lab test)',
 'allergy testing photo patch tests',
 'photo patch tests',
 'Photo patch test(s) (specify number of tests)',
 'PHOTO PATCH TEST SPECIFY NUMBER TSTS',
 'Photo patch test',
 'Photo patch test',
 'Photo patch test (procedure)',
 'Photo patch test',
 'PHOTO PATCH TEST']

In [20]:
dict_final['cui_stys']['C0430578']

{'Diagnostic Procedure'}

In [21]:
dict_final['readme'] = 'Look at the ipynb: AMIA | System Demo '

In [22]:
pkl.dump(dict_final, open('../data/dict_system_demo_may25.pkl', 'wb'))

## For the demo
---------------------

In [32]:
dict_final = pkl.load(open('../data/dict_system_demo_may25.pkl', 'rb'))

In [11]:
mm = MetaMap.get_instance('/data/Software/metamap_2020/public_mm/bin/metamap20') 

In [54]:
def get_metamap_output(txt_):
    output = mm.extract_concepts([txt_], [0])[0]
    return output

In [35]:
def get_longest_def(list_):
    len_ = 0
    def_ = ''
    for each_ in list_:
        if each_.__len__()>len_:
            def_ = each_
            len_ = each_.__len__()
    return def_

In [85]:
inp_txt = 'The patient is widowed and lives alone in [** Location (un) **]'
verbose_flag = False
file_name = 'synthetic_EHR_example.txt'

In [82]:
inp_txt = open(file_name).read()

In [86]:
# output = get_metamap_output(inp_txt)
output = get_metamap_output(inp_txt)

In [89]:
print('=='*50)
print(' '*30+'All extracted SDOH CUIs')
print('=='*50)
op_dict = []
for each_concept in output:
    if each_concept.cui in dict_final['cuis']:
        pos_info = each_concept.pos_info.split(',')[0].replace('[', '')
        pos_info = pos_info.replace(']', '')
        
        start_idx = int(pos_info.split('/')[0])-1
        end_idx = start_idx+int(pos_info.split('/')[1])
        
        print('Extracted Phrase:', inp_txt[start_idx:end_idx])
        print('CUI:', each_concept.cui)
#         print(each_concept.cui, each_concept.cui in dict_final['cuis'])
        print('Definition:', each_concept.preferred_name)
        print('Semantic Type(s):', [x for x in dict_final['cui_stys'][each_concept.cui]])
        
        op_dict.append({'cui': each_concept.cui, 
                        'phrase': inp_txt[start_idx:end_idx],
                        'def': each_concept.preferred_name,
                        'stys': [x for x in dict_final['cui_stys'][each_concept.cui]]
                       })
        
        if verbose_flag:
            print('All Definitions:', dict_final['cui_defs'][each_concept.cui])
            print('Start, End:', (start_idx, end_idx))
        print('--'*50)

                              All extracted SDOH CUIs
Extracted Phrase: lives
CUI: C0595998
Definition: Household composition
Semantic Type(s): ['Finding']
----------------------------------------------------------------------------------------------------
Extracted Phrase: patient
CUI: C0030705
Definition: Patients
Semantic Type(s): ['Patient or Disabled Group']
----------------------------------------------------------------------------------------------------
Extracted Phrase: widowed
CUI: C1510465
Definition: Widow
Semantic Type(s): ['Family Group']
----------------------------------------------------------------------------------------------------
Extracted Phrase: widowed
CUI: C0206275
Definition: Widowhood
Semantic Type(s): ['Finding']
----------------------------------------------------------------------------------------------------
Extracted Phrase: alone
CUI: C0439044
Definition: Living Alone
Semantic Type(s): ['Finding']
-----------------------------------------------------

In [42]:
Counter([dict_final['cui_stys'][x].__len__() for x in dict_final['cuis']])

Counter({1: 197852, 2: 810, 3: 15})

In [44]:
each_concept

ConceptMMI(index='0', mm='MMI', score='3.64', preferred_name='alone - group size', cui='C0679994', semtypes='[grpa]', trigger='["alone"-tx-1-"alone"-adj-0]', location='TX', pos_info='34/5', tree_codes='')

In [57]:
inp_txt[34-1:38]

'alone'

In [90]:
op_dict

[{'cui': 'C0595998',
  'phrase': 'lives',
  'def': 'Household composition',
  'stys': ['Finding']},
 {'cui': 'C0030705',
  'phrase': 'patient',
  'def': 'Patients',
  'stys': ['Patient or Disabled Group']},
 {'cui': 'C1510465',
  'phrase': 'widowed',
  'def': 'Widow',
  'stys': ['Family Group']},
 {'cui': 'C0206275',
  'phrase': 'widowed',
  'def': 'Widowhood',
  'stys': ['Finding']},
 {'cui': 'C0439044',
  'phrase': 'alone',
  'def': 'Living Alone',
  'stys': ['Finding']},
 {'cui': 'C0679994',
  'phrase': 'alone',
  'def': 'alone - group size',
  'stys': ['Group Attribute']}]

## Final Demo
--------------

In [1]:
import ease
import json

In [2]:
# chk = 'The patient is widowed and lives alone in [** Location (un) **]'

In [3]:
# json.load(open('config.json'))['semantic_types'].split(',')

In [2]:
op_sdoh_cuis = ease.get_sdoh_cuis(inp_reference='The patient is widowed and lives alone in [** Location (un) **]', 
                                  config_json='config.json',
                                  print_op=True, 
                                  verbose_flag=False,
                                  ehr_flag=False)

                              Extracted SDOH CUIs
Extracted Phrase: lives
CUI: C0595998
Definition: Household composition
Semantic Type(s): ['Finding']
----------------------------------------------------------------------------------------------------
Extracted Phrase: patient
CUI: C0030705
Definition: Patients
Semantic Type(s): ['Patient or Disabled Group']
----------------------------------------------------------------------------------------------------
Extracted Phrase: widowed
CUI: C1510465
Definition: Widow
Semantic Type(s): ['Family Group']
----------------------------------------------------------------------------------------------------
Extracted Phrase: widowed
CUI: C0206275
Definition: Widowhood
Semantic Type(s): ['Finding']
----------------------------------------------------------------------------------------------------
Extracted Phrase: alone
CUI: C0439044
Definition: Living Alone
Semantic Type(s): ['Finding']
---------------------------------------------------------

In [3]:
op_sdoh_cuis = ease.get_sdoh_cuis(inp_reference='synthetic_EHR_example.txt', 
                                  config_json='config.json',
                                  print_op=True, 
                                  verbose_flag=False,
                                  ehr_flag=True)

                              Extracted SDOH CUIs
Extracted Phrase: husband
CUI: C0242664
Definition: husband
Semantic Type(s): ['Family Group']
----------------------------------------------------------------------------------------------------
Extracted Phrase: lives
CUI: C0595998
Definition: Household composition
Semantic Type(s): ['Finding']
----------------------------------------------------------------------------------------------------
Extracted Phrase: patient
CUI: C0030705
Definition: Patients
Semantic Type(s): ['Patient or Disabled Group']
----------------------------------------------------------------------------------------------------
Extracted Phrase: widowed
CUI: C1510465
Definition: Widow
Semantic Type(s): ['Family Group']
----------------------------------------------------------------------------------------------------
Extracted Phrase: widowed
CUI: C0206275
Definition: Widowhood
Semantic Type(s): ['Finding']
-------------------------------------------------------

In [4]:
op_sdoh_cuis = ease.get_sdoh_cuis(inp_reference='synthetic_EHR_example.txt', 
                                  config_json='config_mental_dys.json',
                                  print_op=True, 
                                  verbose_flag=False,
                                  ehr_flag=True)

                              Extracted SDOH CUIs
Extracted Phrase: depression
CUI: C0344315
Definition: Depressed mood
Semantic Type(s): ['Mental or Behavioral Dysfunction']
----------------------------------------------------------------------------------------------------
Extracted Phrase: depression
CUI: C0011581
Definition: Depressive disorder
Semantic Type(s): ['Mental or Behavioral Dysfunction']
----------------------------------------------------------------------------------------------------
Extracted Phrase: depression
CUI: C0011570
Definition: Mental Depression
Semantic Type(s): ['Mental or Behavioral Dysfunction']
----------------------------------------------------------------------------------------------------
Extracted Phrase: suffering
CUI: C0683278
Definition: Mental Suffering
Semantic Type(s): ['Mental or Behavioral Dysfunction']
----------------------------------------------------------------------------------------------------


In [6]:
op_sdoh_cuis = ease.get_all_cuis(inp_reference='The patient is widowed and lives alone in [** Location (un) **]', 
                                 print_op=True, 
                                 verbose_flag=False,
                                 ehr_flag=False)

                              Extracted SDOH CUIs
Extracted Phrase: lives
CUI: C1634625
Definition: % live
Semantic Type(s): [qlco]
----------------------------------------------------------------------------------------------------
Extracted Phrase: Location
CUI: C1515974
Definition: Anatomic Site
Semantic Type(s): [blor]
----------------------------------------------------------------------------------------------------
Extracted Phrase: lives
CUI: C0595998
Definition: Household composition
Semantic Type(s): [fndg]
----------------------------------------------------------------------------------------------------
Extracted Phrase: lives
CUI: C1548795
Definition: Live - Specimen Condition
Semantic Type(s): [qlco]
----------------------------------------------------------------------------------------------------
Extracted Phrase: Location
CUI: C0450429
Definition: Location
Semantic Type(s): [spco]
---------------------------------------------------------------------------------------

In [7]:
ease.get_data_cui(curr_cui='C1634625', config_json='config.json')

,CUI,LAT,TS,LUI,STT,SUI,ISPREF,AUI,SAUI,SCUI,SDUI,SAB,TTY,CODE,STR,SRL,SUPPRESS,CVF
0,C1634625,ENG,S,L5854315,PF,S6683043,Y,A9402723,2573862019,419749002,None,SNOMEDCT_US,FN,419749002,% live (qualifier value),9,N,NaN
1,C1634625,ENG,P,L1225075,PF,S6683045,Y,A9402725,2577169013,419749002,None,SNOMEDCT_US,PT,419749002,% live,9,N,256.0
2,C1634625,ENG,S,L5854314,PF,S6694961,Y,A9414672,2579979015,419749002,None,SNOMEDCT_US,SY,419749002,Percent live,9,N,NaN


In [15]:
ease.get_data_cui(curr_cui='C0206275', config_json='config.json').head()

,CUI,LAT,TS,LUI,STT,SUI,ISPREF,AUI,SAUI,SCUI,SDUI,SAB,TTY,CODE,STR,SRL,SUPPRESS,CVF
0,C0206275,ENG,P,L0250219,PF,S0328631,N,A0362532,None,M0026900,D017805,MSH,MH,D017805,Widowhood,0,N,256.0
1,C0206275,ENG,P,L0250219,PF,S0328631,N,A0362533,None,None,None,SNM,PT,F-97240,Widowhood,9,N,256.0
2,C0206275,ENG,P,L0250219,PF,S0328631,N,A0362534,None,None,None,SNMI,SY,S-11041,Widowhood,9,N,256.0
3,C0206275,ENG,S,L0405920,PF,S0413576,N,A0471791,None,None,None,RCD,PT,1335.,Widowed,3,N,256.0
4,C0206275,ENG,P,L0250219,VC,S0427397,Y,A0491748,None,None,1124-1123,CSP,PT,1124-1123,widowhood,0,N,256.0


In [11]:
ease.are_they_connected(CUI1='C1634625', 
                        CUI2='C1548795',
                        config_json='config.json')[1]

,CUI1,AUI1,STYPE1,REL,CUI2,AUI2,STYPE2,RELA,RUI,SRUI,SAB,SL,RG,DIR,SUPPRESS,CVF
2,C1634625,None,CUI,RO,C1548795,None,CUI,None,R54598635,None,MTH,MTH,None,None,N,None


In [16]:
ease.get_data_cui(curr_cui='C1548795', config_json='config.json').head()

,CUI,LAT,TS,LUI,STT,SUI,ISPREF,AUI,SAUI,SCUI,SDUI,SAB,TTY,CODE,STR,SRL,SUPPRESS,CVF
0,C1548795,ENG,P,L10787549,PF,S13412386,Y,A20674063,None,None,None,MTH,PN,NOCODE,Live - Specimen Condition,0,N,256
1,C1548795,ENG,S,L1225075,PF,S6318468,Y,A8313836,None,None,None,HL7V2.5,PT,LIVE,Live,0,N,256


In [13]:
ease.are_they_connected(CUI1='C1634625', 
                        CUI2='C0206275',
                        config_json='config.json')

(False, [])